In [1]:
# OPTIONAL: Load the "autoreload" extension so that code can change.
%load_ext autoreload

# OPTIONAL: always reload modules so that as you change code in src, it gets loaded.
%autoreload 2

# Смена рабочей папки.
%cd ..

d:\coding\text-proc-ml


d:\coding\text-proc-ml\.venv\lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


# 2.1.4 распределение Ципфа  
f(rank;s,N) = 1/(Z(s,N)*rank^s)  
rank - порядковый номер слова после сортировки по убыванию частоты  
s - коэффициент скорости убывания вероятности  
N - количество слов  
Z(s,N) = sum(i=1->N)(i^-s) - нормализационная константа.  

In [6]:
from src.calculation.formulas import get_zipf_proba

In [4]:
N = 3
rank_list = [1, 2, 3]
s = 2

In [7]:
proba_list = [get_zipf_proba(rank, s, N) for rank in rank_list]
print(*proba_list, sep=" ")

0.7347 0.1837 0.0816


# 2.2.5 Кросс-энтропия для бинарной классификации

In [9]:
from src.calculation.formulas import get_cross_entropy_for_two_examples

In [20]:
examples_dict = {
    "1": [[0.99, 0.01], [0.5, 0.5]],
    "2": [[0.5, 0.5], [0.99, 0.01]],
    "3": [[0.99, 0.45], [0.65, 0.65]],
    "4": [[0.65, 0.65], [0.99, 0.45]],
    "5": [[0.99, 0.45], [0.99, 0.01]],
    "6": [[0.99, 0.01], [0.99, 0.45]],
}

for key, example_list in examples_dict.items():
    ce_first_example = get_cross_entropy_for_two_examples(example_list[0])
    ce_second_example = get_cross_entropy_for_two_examples(example_list[1])
    print(
        f"{key}: {ce_first_example = }, {ce_second_example = }. Total (first is smaller than second): {ce_first_example < ce_second_example}"
    )

1: ce_first_example = 4.615220521841592, ce_second_example = 1.3862943611198906. Total (first is smaller than second): False
2: ce_first_example = 1.3862943611198906, ce_second_example = 4.615220521841592. Total (first is smaller than second): True
3: ce_first_example = 0.8085580320712731, ce_second_example = 0.8615658321849085. Total (first is smaller than second): True
4: ce_first_example = 0.8615658321849085, ce_second_example = 0.8085580320712731. Total (first is smaller than second): False
5: ce_first_example = 0.8085580320712731, ce_second_example = 4.615220521841592. Total (first is smaller than second): True
6: ce_first_example = 4.615220521841592, ce_second_example = 0.8085580320712731. Total (first is smaller than second): False


# 2.3.2 Производная функции потерь кросс-энтропии

In [31]:
import sympy.parsing.sympy_parser

import sympy

import math

x*(-y*exp(b + w*x) - y + exp(b + w*x))/(exp(b + w*x) + 1)


In [53]:
x, y, w, b, c = sympy.symbols("x y w b c")
log_res_formula = -y * sympy.log(1 / (1 + sympy.exp(-w * x - b))) - (1 - y) * sympy.log(
    1 - 1 / (1 + sympy.exp(-w * x - b))
)
log_res_formula

-y*log(1/(exp(-b - w*x) + 1)) - (1 - y)*log(1 - 1/(exp(-b - w*x) + 1))

In [ ]:
# производная по w
res_w = sympy.simplify(sympy.diff(log_res_formula, w))
display(res_w)
print(str(res_w))

x*(-y*exp(b + w*x) - y + exp(b + w*x))/(exp(b + w*x) + 1)

x*(-y*exp(b + w*x) - y + exp(b + w*x))/(exp(b + w*x) + 1)


In [58]:
# производная по b
res_b = sympy.simplify(sympy.diff(log_res_formula, b))
display(res_b)
print(str(sympy.simplify(res_b)))

(-y*exp(b + w*x) - y + exp(b + w*x))/(exp(b + w*x) + 1)

(-y*exp(b + w*x) - y + exp(b + w*x))/(exp(b + w*x) + 1)


In [ ]:
# производная по w с l2-регуляризацией
l2_reg = c * (w**2 + b**2)
loss_l2 = log_res_formula + l2_reg
display(loss_l2)

c*(b**2 + w**2) - y*log(1/(exp(-b - w*x) + 1)) - (1 - y)*log(1 - 1/(exp(-b - w*x) + 1))

In [63]:
res_w_l2 = sympy.simplify(sympy.diff(loss_l2, w))
display(res_w_l2)
print(str(res_w_l2))

(2*c*w*exp(b + w*x) + 2*c*w - x*y*exp(b + w*x) - x*y + x*exp(b + w*x))/(exp(b + w*x) + 1)

(2*c*w*exp(b + w*x) + 2*c*w - x*y*exp(b + w*x) - x*y + x*exp(b + w*x))/(exp(b + w*x) + 1)


In [50]:
import sympy.parsing.sympy_parser

sample_expr_str = str(res_w_l2)
sample_expr = sympy.parsing.sympy_parser.parse_expr(sample_expr_str)
sample_value = sample_expr.evalf(subs=dict(x=0.5, y=1, w=4, b=1, c=1))
print(sample_value)

7.97628706341122


In [ ]:
# Получаем обновления веса с помощью градиентного спуска.
# learning rate
t = sympy.symbols("t")

# Обновленный вес - старый вес (w) минус градиент на learning_rate
new_w = w - t * res_w_l2
display(new_w)
print(new_w)

-t*(2*c*w*exp(b + w*x) + 2*c*w - x*y*exp(b + w*x) - x*y + x*exp(b + w*x))/(exp(b + w*x) + 1) + w

-t*(2*c*w*exp(b + w*x) + 2*c*w - x*y*exp(b + w*x) - x*y + x*exp(b + w*x))/(exp(b + w*x) + 1) + w


# 2.3.7 точечная взаимная информация

In [108]:
from src.calculation.pmi import calculate_pmi
import numpy as np

In [84]:
a = "1 0 0 1 1 0"
b = "1 0 0 0 1 0"
pmi_value = calculate_pmi(a, b)

print("{:.6f}".format(pmi_value))

a = '1 0 0 1 1 0',
b = '1 0 0 0 1 0'
0.000000


In [94]:
a = [1, 0, 0, 1, 1, 0]
b = [1, 0, 0, 0, 1, 0]

In [113]:
def get_single_proba(x):
    return sum(x) / len(x)


def get_combine_proba(x, y):
    assert len(a) == len(b)
    return sum([x and y for x, y in zip(a, b)]) / len(a)


pmi = np.log(get_combine_proba(x, y) / (get_single_proba(a) * get_single_proba(b)))
float(round(pmi, 6))

0.693147

# 2.3.8 Встречаемость по Ципфу.

In [139]:
f = lambda rank, s, n: 1 / (rank**s * sum([i**-s for i in range(1, n + 1)]))
n = 1000
target = 0.001
probas = [f(i, s, n) for i in range(1, n + 1)]
len(list(filter(lambda p: p < target, probas)))

976